In [4]:
## 1. Preparación e Importación de Librerías
from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import RDF, DCTERMS, DC, SKOS, XSD, OWL
import pandas as pd
import os   
import unicodedata
import re

# 1.1. Dominio Modificado a example.org
domain = 'https://valencia.example.org/' 

# --- FUNCIONES AUXILIARES ---

# Función para crear slugs seguros (URIs limpias)
def slugify(text):
    text = unicodedata.normalize("NFD", text)
    text = text.encode("ascii", "ignore").decode("utf-8")
    text = re.sub(r"[^a-zA-Z0-9\-]", "-", text)
    text = re.sub(r"-+", "-", text)
    return text.lower().strip("-")

# Función de normalización para coincidencia robusta (quita tildes, mayúsculas)
def normalize_for_match(text):
    if not isinstance(text, str): return ""
    text = unicodedata.normalize("NFD", text)
    text = text.encode("ascii", "ignore").decode("utf-8")
    return text.upper().strip()

# --- MAPEO EXPLÍCITO: Diccionario manual de Wikidata ---
# Mapeo manual de nombres de distritos (CSV) a QIDs de Wikidata
WIKIDATA_MAPPING = {
    "CIUTAT VELLA": "Q3392733", # Ciutat Vella
    "L'EIXAMPLE": "Q3392791",   # Ensanche/Eixample
    "EXTRAMURS": "Q3393056",      # Extramurs
    "CAMPANAR": "Q3393032",     # Campanar
    "LA SAIDIA": "Q3393007",   # La Saïdia
    "EL PLA DEL REAL": "Q4452090",# Pla del Real
    "L'OLIVERETA": "Q3392792",  # L'Olivereta
    "PATRAIX": "Q55226",      # Patraix
    "JESUS": "Q3393002",        # Jesús
    "QUATRE CARRERES": "Q3393088",# Quatre Carreres
    "POBLATS MARITIMS": "Q3392780",# Poblats Marítims
    "CAMINS AL GRAU": "Q3392782",# Camins al Grau
    "ALGIROS": "Q3392835",     # Algiros
    "BENIMACLET": "Q536707",   # Benimaclet
    "RASCANYA": "Q3392759",     # Rascanya
    "BENICALAP": "Q777986",   # Benicalap
    "POBLATS DEL NORD": "Q3393120",# Poblados del Norte
    "POBLATS DE L'OEST": "Q3393115",# Poblados del Oeste
    "POBLATS DEL SUD": "Q3392798",  # Poblados del Sur
}
# Normalizamos las claves del manual para asegurar coincidencias
WIKIDATA_MAPPING = {normalize_for_match(k): v for k, v in WIKIDATA_MAPPING.items()}


# Instancia el grafo y los namespaces
g = Graph()
g.bind("rdf", RDF)
g.bind("dcterms", DCTERMS)
g.bind("dc", DC)
g.bind("skos", SKOS)
g.bind("xsd", XSD)
g.bind("owl", OWL)
schema = Namespace("https://schema.org/")
g.bind("schema", schema)
# Namespace para GeoSPARQL (opcional pero recomendado para WKT)
GEO = Namespace("http://www.opengis.net/ont/geosparql#")
g.bind("geo", GEO)


## 2. Lectura y Procesamiento del CSV (Solo Distritos)

print("--- Cargando y procesando: districtes-distritos.csv ---")

try:
    # Usamos el CSV que has subido
    df_distritos = pd.read_csv ('districtes-distritos.csv', sep=';')
    print(f"CSV cargado con éxito. Total de filas: {len(df_distritos)}")
except FileNotFoundError:
    print("Error: Asegúrate de que el archivo 'districtes-distritos.csv' está en la ubicación correcta.")
    exit()


## 3. Transformación a RDF: Creación de 19 Recursos schema:Place

for index, row in df_distritos.iterrows():
    # Sanitiza el nombre y código
    nombre_distrito = row["Nombre"].strip()
    
    # Normalizamos el nombre del CSV para buscarlo en el diccionario
    nombre_key = normalize_for_match(nombre_distrito)
    
    # --- FIX: Filtrar filas inválidas (basura en el CSV) ---
    if nombre_key not in WIKIDATA_MAPPING:
        print(f"Saltando fila inválida o no encontrada en Wikidata: {nombre_distrito[:30]}...")
        continue
    # -------------------------------------------------------

    codigo_distrito = str(row["Código distrito"]).strip()
    
    # Crea el URI único para el recurso schema:Place usando slugify
    place_uri_name = slugify(nombre_distrito)
    place_uri = URIRef(domain + 'district/' + place_uri_name)
    
    # 3.1. Triples Básicas del schema:Place
    g.add((place_uri, RDF.type, schema.Place))
    g.add((place_uri, RDF.type, SKOS.Concept)) # Buena práctica
    
    # Etiquetas con idioma
    g.add((place_uri, schema.name, Literal(nombre_distrito, lang="ca"))) # Asumimos catalán por el CSV
    g.add((place_uri, SKOS.prefLabel, Literal(nombre_distrito, lang="ca")))
    
    # Identificador
    g.add((place_uri, schema.identifier, Literal(codigo_distrito)))

    # Obtenemos el QID usando la clave normalizada
    qid = WIKIDATA_MAPPING.get(nombre_key)
    if qid:
        wikidata_uri = URIRef(f"http://www.wikidata.org/entity/{qid}")
        # Enlazamos con schema:sameAs y owl:sameAs para máxima interoperabilidad
        g.add((place_uri, schema.sameAs, wikidata_uri)) 
        g.add((place_uri, OWL.sameAs, wikidata_uri))

    
    # 3.2. Enriquecimiento con GeoShape (Polígono)
    if pd.notnull(row["geo_shape"]):
        shape_uri = URIRef(place_uri + "/geoshape")
        g.add((place_uri, schema.geo, shape_uri))
        g.add((shape_uri, RDF.type, schema.GeoShape))
        # Usamos geo:wktLiteral para el polígono (GeoSPARQL standard)
        # Nota: Asumimos que el formato en CSV es compatible con WKT o GeoJSON. 
        # Si es GeoJSON, schema:polygon es correcto como string. Si es WKT, usar GEO.wktLiteral.
        # Dado que parece GeoJSON en el CSV original, lo mantenemos como Literal simple o string, 
        # pero si fuera WKT usaríamos datatype=GEO.wktLiteral.
        g.add((shape_uri, schema.polygon, Literal(row["geo_shape"]))) 

    # 3.3. Enriquecimiento con GeoCoordinates (Punto Central)
    if pd.notnull(row["geo_point_2d"]):
        lat_lon = row["geo_point_2d"].split(',')
        lat = lat_lon[0].strip()
        lon = lat_lon[1].strip()
        
        geo_coord = URIRef(place_uri + "/center")
        
        # Usamos schema:geo
        g.add((place_uri, schema.geo, geo_coord)) 
        
        g.add((geo_coord, RDF.type, schema.GeoCoordinates))
        # Usamos XSD.decimal
        g.add((geo_coord, schema.latitude, Literal(lat, datatype=XSD.decimal)))
        g.add((geo_coord, schema.longitude, Literal(lon, datatype=XSD.decimal)))


## 4. Serialización del Grafo
# Guardamos el resultado en formato Turtle (ttl)
output_dir = "rdf" # Carpeta en el directorio actual
output_file_name = "valencia_districts_places_enriched.ttl"
output_file = os.path.join(output_dir, output_file_name)

# Crea la carpeta si no existe
os.makedirs(output_dir, exist_ok=True) 

g.serialize(destination=output_file)

print("\n--- Resultado Final ---")
print(f"Grafo RDF generado y guardado en: {output_file}")
print(f"Total de triples generadas: {len(g)}")

# Imprime un ejemplo de triple generado para visualización (opcional)
print("\nEjemplo de Triples generados para CIUTAT VELLA (con schema:sameAs):")
example_uri = URIRef(domain + 'district/ciutat-vella')
count = 0
for s, p, o in g.triples((example_uri, None, None)):
    print(s, p, o)
    count += 1
    if count >= 15:
        break

--- Cargando y procesando: districtes-distritos.csv ---
CSV cargado con éxito. Total de filas: 19

--- Resultado Final ---
Grafo RDF generado y guardado en: rdf\valencia_districts_places_enriched.ttl
Total de triples generadas: 266

Ejemplo de Triples generados para CIUTAT VELLA (con schema:sameAs):
https://valencia.example.org/district/ciutat-vella http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://schema.org/Place
https://valencia.example.org/district/ciutat-vella http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2004/02/skos/core#Concept
https://valencia.example.org/district/ciutat-vella https://schema.org/name CIUTAT VELLA
https://valencia.example.org/district/ciutat-vella http://www.w3.org/2004/02/skos/core#prefLabel CIUTAT VELLA
https://valencia.example.org/district/ciutat-vella https://schema.org/identifier 1
https://valencia.example.org/district/ciutat-vella https://schema.org/sameAs http://www.wikidata.org/entity/Q3392733
https://valencia.example.org/distr

In [ ]:
# Instrucciones para visualizar los datos en Wikidata Query Service:
# 1. Abre https://query.wikidata.org/
# 2. Copia y pega la siguiente consulta SPARQL en el editor.
# 3. Ejecuta la consulta (botón "Play" o Ctrl+Enter).

# Esta consulta selecciona y muestra en un mapa los distritos de Valencia que hemos enlazado.

#defaultView:Map
SELECT ?r ?rLabel ?location
WHERE {
  # VALUES define una lista fija de elementos (nuestros distritos por su ID de Wikidata - QID)
  VALUES ?r { 
    wd:Q3392835 wd:Q777986 wd:Q536707 wd:Q3392782 wd:Q3393032 
    wd:Q3392733 wd:Q4452090 wd:Q3393056 wd:Q3393002 wd:Q3392791 
    wd:Q3392792 wd:Q3393007 wd:Q55226 wd:Q3393115 wd:Q3393120 
    wd:Q3392798 wd:Q3392780 wd:Q3393088 wd:Q3392759 
  }
  
  ?r wdt:P625 ?location. # Obtiene la propiedad P625 (coordenadas geográficas) de cada distrito (?r)
  
  # Servicio de etiquetas para obtener los nombres de los distritos automáticamente
  SERVICE wikibase:label { 
    bd:serviceParam wikibase:language "[AUTO_LANGUAGE],es". # Intenta en el idioma del navegador, si no, español
    ?r rdfs:label ?rLabel. # Asigna la etiqueta a la variable ?rLabel
  }
}